## Code for running inference with pre-trained models and training from scratch (on GPU)

Running inference with pre-trained models will replicate the results reported in the paper.

Training from scratch may have some variations due to inherent randomness.

This code is based on Tensorflow. If you want to use HuggingFace code, please refer to this [Colab link](https://colab.research.google.com/drive/1NO228tvSJWDVI3Lj506TOB4HxZbm4ftL?usp=sharing)

##Install Requirements

In [ ]:
!pip install t5
!pip install ipdb
!pip install --upgrade gdown
!git clone https://github.com/dair-iitd/moie.git
!pip install overrides
!pip install pytorch-lightning==0.7.6
!pip install scikit-learn==0.23.1
!pip install torchtext==0.5.0
!pip install transformers==2.6.0
!pip install tokenizers==0.5.2
!pip install docopt

import nltk
nltk.download('stopwords')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.5/164.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.2/385.2 kB 25.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.2/347.2 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 110.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.0/537.0 kB 49.4 MB/s eta 0:00

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
%cd /content/moie/code

/content/moie/code


##Train and Evaluate models on various languages
INPUT : path to file containing sentences from which OpenIE triples have to be extracted

OUTPUT : path to output file containing the final extractions

Note that during evaluation a lot of intermediate files are also generated using the prefix ${OUTPUT}.*

###ENGLISH Training

In [ ]:
%%shell

DEVICE=0 ## GPU device number 0

mkdir -p data
gdown 1u3YUgAjVAjGCckVdYZgKhkcn_ninkXo-
unzip en_gen2oie_s2.zip
mv root/en_gen2oie_s2 data
gdown 1oE11PYPS497HuhdXLn54JIDgOwQhX4Ab
unzip en_gen2oie_s1.zip
mv root/en_gen2oie_s1 data
rm -r root

bash train.sh en_gen2oie_s1 models/en_gen2oie_s1_scratch data $DEVICE 10000
bash train.sh en_gen2oie_s2 models/en_gen2oie_s2_scratch data $DEVICE 10000

## Uncomment below lines for running on TPU
# DEVICE=$TPU_NAME
# bash train.sh en_gen2oie_s1 gs://BUCKET_NAME/models/en_gen2oie_s1_scratch gs://BUCKET_NAME/data $DEVICE 10000
# bash train.sh en_gen2oie_s2 gs://BUCKET_NAME/models/en_gen2oie_s2_scratch gs://BUCKET_NAME/data $DEVICE 10000

### ENGLISH Evaluation
Result with pre-trained model: (54.5 F1, 38.9 AUC)


In [ ]:
%%shell
gdown 172Yelz3dQggL173Pwwkwd-ex-N84K0zu

Downloading...
From: https://drive.google.com/uc?id=172Yelz3dQggL173Pwwkwd-ex-N84K0zu
To: /content/en_label_rescore.ckpt
100% 2.13G/2.13G [00:41<00:00, 51.8MB/s]


In [ ]:
ls

In [ ]:
%%shell
INPUT=carb/data/en_test.input
OUTPUT=en_test2.output
DEVICE_NAME=0

# Download trained models
mkdir -p models
gdown 101Ck5QVztW5BqOwIARFTACwGI7gHcegU # gen2oie_s2
unzip en_gen2oie_s2.zip
mv root/en_gen2oie_s2 models/
rm -r root
gdown 1uqk3vAce7ESTdfilT_wUBPfUY8xC8rR6 # gen2oie_s1
unzip en_gen2oie_s1.zip
mv root/en_gen2oie_s1 models/
rm -r root
mkdir -p models/en_label_rescore # label_rescore
gdown 172Yelz3dQggL173Pwwkwd-ex-N84K0zu
mv en_label_rescore.ckpt models/en_label_rescore

bash gen2oie_evaluate.sh en models/en_gen2oie_s1 models/en_gen2oie_s2 models/en_label_rescore ${DEVICE_NAME} ${INPUT} ${OUTPUT}
python carb/carb.py --gold carb/data/gold/en_test.tsv --out /dev/null --allennlp ${OUTPUT}.carb

# Uncomment below two lines for running on TPU
# DEVICE_NAME=${TPU_NAME}
# bash gen2oie_evaluate.sh en gs://moie_bucket/models/en_gen2oie_s1 gs://moie_bucket/models/en_gen2oie_s2 dummy ${DEVICE_NAME} ${INPUT} ${OUTPUT}
# python carb/carb.py --gold carb/data/gold/en_test.tsv --out /dev/null --allennlp ${OUTPUT}.carb

Downloading...
From (uriginal): https://drive.google.com/uc?id=101Ck5QVztW5BqOwIARFTACwGI7gHcegU
From (redirected): https://drive.google.com/uc?id=101Ck5QVztW5BqOwIARFTACwGI7gHcegU&confirm=t&uuid=0718a408-dc0e-4244-ba51-93618509c6b0
To: /content/moie/code/en_gen2oie_s2.zip
100% 930M/930M [00:11<00:00, 83.0MB/s]
Archive:  en_gen2oie_s2.zip
  inflating: root/en_gen2oie_s2/checkpoint  
   creating: root/en_gen2oie_s2/commands/
  inflating: root/en_gen2oie_s2/commands/command.5  
  inflating: root/en_gen2oie_s2/commands/command.3  
  inflating: root/en_gen2oie_s2/commands/command.1  
  inflating: root/en_gen2oie_s2/commands/command.7  
  inflating: root/en_gen2oie_s2/commands/command.2  
  inflating: root/en_gen2oie_s2/commands/command.4  
  inflating: root/en_gen2oie_s2/commands/command  
  inflating: root/en_gen2oie_s2/commands/command.6  
  inflating: root/en_gen2oie_s2/model.ckpt-1010000.data-00000-of-00002  
  inflating: root/en_gen2oie_s2/model.ckpt-1010000.data-00001-of-00002  
  in

CalledProcessError: ignored

In [ ]:
#pip install scikit-learn==0.18.0
#pip install filepath\scikit_learn‑0.18.0‑cp310‑cp310‑win_amd64.whl
#!pip install scikit_learn‑0.18.0‑cp39‑cp39‑win_amd64.whl
!pipwin install scikit-learn

/bin/bash: pipwin: command not found


In [ ]:
%%shell
bash gen2oie_evaluate.sh en models/en_gen2oie_s1 models/en_gen2oie_s2 models/en_label_rescore ${DEVICE_NAME} ${INPUT} ${OUTPUT}
python carb/carb.py --gold carb/data/gold/en_test.tsv --out /dev/null --allennlp ${OUTPUT}.carb

###SPANISH Training

In [ ]:
%%shell

DEVICE=0 ## GPU device number 0

mkdir -p data
gdown 1lZfvoYUOk7AUvUXGVn54ePzEqBged1Xf
unzip es_gen2oie_s2_aact_moie.zip
mv root/es_gen2oie_s2_aact_moie data
gdown 13YUPpHP86zxN54w7dZvBxqEWndFPNvY9
unzip es_gen2oie_s1_aact_moie.zip
mv root/es_gen2oie_s1_aact_moie data
rm -r root

bash train.sh es_gen2oie_s1_aact_moie models/es_gen2oie_s1_aact_moie_scratch data $DEVICE 10000
bash train.sh es_gen2oie_s2_aact_moie models/es_gen2oie_s2_aact_moie_scratch data $DEVICE 10000

## Uncomment below lines for running on TPU
# DEVICE=$TPU_NAME
# bash train.sh es_gen2oie_s1_aact_moie gs://BUCKET_NAME/models/es_gen2oie_s1_aact_moie_scratch gs://BUCKET_NAME/data $DEVICE 10000
# bash train.sh es_gen2oie_s2_aact_moie gs://BUCKET_NAME/models/es_gen2oie_s2_aact_moie_scratch gs://BUCKET_NAME/data $DEVICE 10000

### SPANISH Evaluation
Result with pre-trained model: (68.04 F1, 48.59 AUC)


In [ ]:
%%shell
LANGUAGE=es
INPUT=carb/data/${LANGUAGE}_test.input
OUTPUT=${LANGUAGE}_test.output
DEVICE_NAME=0 ## GPU device number 0

## Download trained models
mkdir -p models
gdown 1vL6Ur45NoXl-Im1gsQ55B0uwGgHI01qj  # gen2oie_s2
unzip ${LANGUAGE}_gen2oie_s2_aact_moie.zip
mv root/${LANGUAGE}_gen2oie_s2_aact_moie models/
rm -r root
gdown 1Ij5cf94BUL4uITfCSc4iBaO6XvRIC1GH # gen2oie_s1
unzip ${LANGUAGE}_gen2oie_s1_aact_moie.zip
mv root/${LANGUAGE}_gen2oie_s1_aact_moie models/
rm -r root
mkdir -p models/${LANGUAGE}_label_rescore # label_rescore
gdown 12ak3gGLAW3-ZSJP5zgSZgFftGubNPyBh
mv ${LANGUAGE}_label_rescore.ckpt models/${LANGUAGE}_label_rescore

bash gen2oie_evaluate.sh ${LANGUAGE} models/${LANGUAGE}_gen2oie_s1_aact_moie models/${LANGUAGE}_gen2oie_s2_aact_moie models/${LANGUAGE}_label_rescore ${DEVICE_NAME} ${INPUT} ${OUTPUT}
python carb/carb.py --gold carb/data/gold/${LANGUAGE}_test.tsv --out /dev/null --allennlp ${LANGUAGE}_test.output.carb

## Run on TPU
# bash gen2oie_evaluate.sh ${LANGUAGE} gs://moie_bucket/models/${LANGUAGE}_gen2oie_s1_aact_moie gs://moie_bucket/models/${LANGUAGE}_gen2oie_s2_aact_moie models/${LANGUAGE}_label_rescore $TPU_NAME ${INPUT} ${OUTPUT}


Downloading...
From: https://drive.google.com/uc?id=1vL6Ur45NoXl-Im1gsQ55B0uwGgHI01qj
To: /content/moie/es_gen2oie_s2_aact_moie.zip
100% 930M/930M [00:03<00:00, 280MB/s]
Archive:  es_gen2oie_s2_aact_moie.zip
  inflating: root/es_gen2oie_s2_aact_moie/checkpoint  
   creating: root/es_gen2oie_s2_aact_moie/commands/
  inflating: root/es_gen2oie_s2_aact_moie/commands/command.3  
  inflating: root/es_gen2oie_s2_aact_moie/commands/command.1  
  inflating: root/es_gen2oie_s2_aact_moie/commands/command.2  
  inflating: root/es_gen2oie_s2_aact_moie/commands/command  
  inflating: root/es_gen2oie_s2_aact_moie/model.ckpt-1010000.data-00000-of-00002  
  inflating: root/es_gen2oie_s2_aact_moie/model.ckpt-1010000.data-00001-of-00002  
  inflating: root/es_gen2oie_s2_aact_moie/model.ckpt-1010000.index  
  inflating: root/es_gen2oie_s2_aact_moie/model.ckpt-1010000.meta  
  inflating: root/es_gen2oie_s2_aact_moie/operative_config.gin  
mv: cannot move 'root/es_gen2oie_s2_aact_moie' to 'models/es_gen2oi

###PORTUGUESE Training

In [ ]:
%%shell

DEVICE=0 ## GPU device number 0

mkdir -p data
gdown 16ri4JU0Xyrn3OhPSG3vJNIU-2dXpudpP
unzip pt_gen2oie_s2_aact_moie.zip
mv root/pt_gen2oie_s2_aact_moie data
gdown 1aRs3M6TkCRIo_M31vNPn7-xO5wCrzQ0Q
unzip pt_gen2oie_s1_aact_moie.zip
mv root/pt_gen2oie_s1_aact_moie data
rm -r root

bash train.sh pt_gen2oie_s1_aact_moie models/pt_gen2oie_s1_aact_moie_scratch data $DEVICE 10000
bash train.sh pt_gen2oie_s2_aact_moie models/pt_gen2oie_s2_aact_moie_scratch data $DEVICE 10000

## Uncomment below lines for running on TPU
# DEVICE=$TPU_NAME
# bash train.sh pt_gen2oie_s1_aact_moie gs://BUCKET_NAME/models/pt_gen2oie_s1_aact_moie_scratch gs://BUCKET_NAME/data $DEVICE 10000
# bash train.sh pt_gen2oie_s2_aact_moie gs://BUCKET_NAME/models/pt_gen2oie_s2_aact_moie_scratch gs://BUCKET_NAME/data $DEVICE 10000

### PORTUGUESE Evaluation
Result with pre-trained model: (66.39 F1, 49.12 AUC)


In [ ]:
%%shell
LANGUAGE=pt
INPUT=carb/data/${LANGUAGE}_test.input
OUTPUT=${LANGUAGE}_test.output
DEVICE_NAME=0 ## GPU device number 0

## Download trained models
mkdir -p models
gdown 1U_E8lEtZuqUsgauC2pBWc-DzhiOcJJYA # gen2oie_s2
unzip ${LANGUAGE}_gen2oie_s2_aact_moie.zip
mv root/${LANGUAGE}_gen2oie_s2_aact_moie models/
rm -r root
gdown 1ypz9r_LR3rLhcVBczaWHCjbofH6XLNza # gen2oie_s1
unzip ${LANGUAGE}_gen2oie_s1_aact_moie.zip
mv root/${LANGUAGE}_gen2oie_s1_aact_moie models/
rm -r root
mkdir -p models/${LANGUAGE}_label_rescore # label_rescore
gdown 15HJMWaw7H0ndKotoYIZpg46rr3A9nFuy
mv ${LANGUAGE}_label_rescore.ckpt models/${LANGUAGE}_label_rescore

bash gen2oie_evaluate.sh ${LANGUAGE} models/${LANGUAGE}_gen2oie_s1_aact_moie models/${LANGUAGE}_gen2oie_s2_aact_moie models/${LANGUAGE}_label_rescore ${DEVICE_NAME} ${INPUT} ${OUTPUT}
python carb/carb.py --gold carb/data/gold/${LANGUAGE}_test.tsv --out /dev/null --allennlp ${LANGUAGE}_test.output.carb

## Run on TPU
# bash gen2oie_evaluate.sh ${LANGUAGE} gs://moie_bucket/models/${LANGUAGE}_gen2oie_s1_aact_moie gs://moie_bucket/models/${LANGUAGE}_gen2oie_s2_aact_moie models/${LANGUAGE}_label_rescore $TPU_NAME ${INPUT} ${OUTPUT}


Downloading...
From: https://drive.google.com/uc?id=1U_E8lEtZuqUsgauC2pBWc-DzhiOcJJYA
To: /content/moie/pt_gen2oie_s2_aact_moie.zip
100% 930M/930M [00:09<00:00, 103MB/s]
Archive:  pt_gen2oie_s2_aact_moie.zip
  inflating: root/pt_gen2oie_s2_aact_moie/checkpoint  
   creating: root/pt_gen2oie_s2_aact_moie/commands/
  inflating: root/pt_gen2oie_s2_aact_moie/commands/command.1  
  inflating: root/pt_gen2oie_s2_aact_moie/commands/command  
  inflating: root/pt_gen2oie_s2_aact_moie/model.ckpt-1010000.data-00000-of-00002  
  inflating: root/pt_gen2oie_s2_aact_moie/model.ckpt-1010000.data-00001-of-00002  
  inflating: root/pt_gen2oie_s2_aact_moie/model.ckpt-1010000.index  
  inflating: root/pt_gen2oie_s2_aact_moie/model.ckpt-1010000.meta  
  inflating: root/pt_gen2oie_s2_aact_moie/operative_config.gin  
Downloading...
From: https://drive.google.com/uc?id=1ypz9r_LR3rLhcVBczaWHCjbofH6XLNza
To: /content/moie/pt_gen2oie_s1_aact_moie.zip
100% 930M/930M [00:09<00:00, 95.5MB/s]
Archive:  pt_gen2oie_s

###HINDI Training

In [ ]:
%%shell

DEVICE=0 ## GPU device number 0

mkdir -p data
gdown 1wR0Rrstav8yoRi1pzyeOwv86ixNH4nUm
unzip hi_gen2oie_s2_aact_moie.zip
mv root/hi_gen2oie_s2_aact_moie data
gdown 1NbspTQHIQXvJVyBP-BG6BsleRQwnpwcO
unzip hi_gen2oie_s1_aact_moie.zip
mv root/hi_gen2oie_s1_aact_moie data
rm -r root

bash train.sh hi_gen2oie_s1_aact_moie models/hi_gen2oie_s1_aact_moie_scratch data $DEVICE 10000
bash train.sh hi_gen2oie_s2_aact_moie models/hi_gen2oie_s2_aact_moie_scratch data $DEVICE 10000

## Uncomment below lines for running on TPU
# DEVICE=$TPU_NAME
# bash train.sh hi_gen2oie_s1_aact_moie gs://BUCKET_NAME/models/hi_gen2oie_s1_aact_moie_scratch gs://BUCKET_NAME/data $DEVICE 10000
# bash train.sh hi_gen2oie_s2_aact_moie gs://BUCKET_NAME/models/hi_gen2oie_s2_aact_moie_scratch gs://BUCKET_NAME/data $DEVICE 10000

### HINDI Evaluation
Result with pre-trained model: (53.54 F1, 30.9 AUC)


In [ ]:
%%shell
LANGUAGE=hi
INPUT=carb/data/${LANGUAGE}_test.input
OUTPUT=${LANGUAGE}_test.output
DEVICE_NAME=0 ## GPU device number 0

## Download trained models
mkdir -p models
gdown 1mjqtwi2V-sQDHXV43E9-LOYUSrszDgi4 #s2
unzip ${LANGUAGE}_gen2oie_s2_aact_moie.zip
mv root/${LANGUAGE}_gen2oie_s2_aact_moie models/
rm -r root
gdown 1VKQr8blOfv_9UklyZRmHAewjYP_jx-R4 #s1
unzip ${LANGUAGE}_gen2oie_s1_aact_moie.zip
mv root/${LANGUAGE}_gen2oie_s1_aact_moie models/
rm -r root
mkdir -p models/${LANGUAGE}_label_rescore # label_rescore
gdown 15Z7Km2KKXTmcsMmZ31wGtUtrowk_3wKi
mv ${LANGUAGE}_label_rescore.ckpt models/${LANGUAGE}_label_rescore

bash gen2oie_evaluate.sh ${LANGUAGE} models/${LANGUAGE}_gen2oie_s1_aact_moie models/${LANGUAGE}_gen2oie_s2_aact_moie models/${LANGUAGE}_label_rescore ${DEVICE_NAME} ${INPUT} ${OUTPUT}
python carb/carb.py --gold carb/data/gold/${LANGUAGE}_test.tsv --out /dev/null --allennlp ${LANGUAGE}_test.output.carb

## Run on TPU
# bash gen2oie_evaluate.sh ${LANGUAGE} gs://moie_bucket/models/${LANGUAGE}_gen2oie_s1_aact_moie gs://moie_bucket/models/${LANGUAGE}_gen2oie_s2_aact_moie models/${LANGUAGE}_label_rescore $TPU_NAME ${INPUT} ${OUTPUT}


###TELUGU Training

In [ ]:
%%shell

DEVICE=0 ## GPU device number 0

mkdir -p data
gdown 1qksq4Wcy9D6JG4zW_XyQ7opl2nizddiK
unzip te_gen2oie_s2_aact_moie.zip
mv root/te_gen2oie_s2_aact_moie data
gdown 1NG7iods5LPB4n309n1kW8jHSQrHhdIEg
unzip te_gen2oie_s1_aact_moie.zip
mv root/te_gen2oie_s1_aact_moie data
rm -r root

bash train.sh te_gen2oie_s1_aact_moie models/te_gen2oie_s1_aact_moie_scratch data $DEVICE 10000
bash train.sh te_gen2oie_s2_aact_moie models/te_gen2oie_s2_aact_moie_scratch data $DEVICE 10000

## Uncomment below lines for running on TPU
# DEVICE=$TPU_NAME
# bash train.sh te_gen2oie_s1_aact_moie gs://BUCKET_NAME/models/te_gen2oie_s1_aact_moie_scratch gs://BUCKET_NAME/data $DEVICE 10000
# bash train.sh te_gen2oie_s2_aact_moie gs://BUCKET_NAME/models/te_gen2oie_s2_aact_moie_scratch gs://BUCKET_NAME/data $DEVICE 10000

### TELUGU Evaluation
Result with pre-trained model: (39.85 F1, 16.12 AUC)


In [ ]:
%%shell
LANGUAGE=te
INPUT=carb/data/${LANGUAGE}_test.input
OUTPUT=${LANGUAGE}_test.output
DEVICE_NAME=0 ## GPU device number 0

## Download trained models
mkdir -p models
gdown 1Ng9SifgcdKk6yVTOyjI0EC3NIgVHHZ-M
unzip ${LANGUAGE}_gen2oie_s2_aact_moie.zip
mv root/${LANGUAGE}_gen2oie_s2_aact_moie models/
rm -r root
gdown 1awqVto_sxju57T9Eg5wh7fP5pU5Ftwaj
unzip ${LANGUAGE}_gen2oie_s1_aact_moie.zip
mv root/${LANGUAGE}_gen2oie_s1_aact_moie models/
rm -r root
mkdir -p models/${LANGUAGE}_label_rescore # label_rescore
gdown 1cm_Xfx_7mg87NYB7y1yHfQUwMnZUqBOB
mv ${LANGUAGE}_label_rescore.ckpt models/${LANGUAGE}_label_rescore

bash gen2oie_evaluate.sh ${LANGUAGE} models/${LANGUAGE}_gen2oie_s1_aact_moie models/${LANGUAGE}_gen2oie_s2_aact_moie models/${LANGUAGE}_label_rescore ${DEVICE_NAME} ${INPUT} ${OUTPUT}
python carb/carb.py --gold carb/data/gold/${LANGUAGE}_test.tsv --out /dev/null --allennlp ${LANGUAGE}_test.output.carb

## Run on TPU
# bash gen2oie_evaluate.sh ${LANGUAGE} gs://moie_bucket/models/${LANGUAGE}_gen2oie_s1_aact_moie gs://moie_bucket/models/${LANGUAGE}_gen2oie_s2_aact_moie models/${LANGUAGE}_label_rescore $TPU_NAME ${INPUT} ${OUTPUT}


###CHINESE Training

In [ ]:
%%shell

DEVICE=0 ## GPU device number 0

mkdir -p data
gdown 1ry6xEvwjHKmYaxrqcbt46pdO96zyIuP-
unzip zh_gen2oie_s2_aact_moie.zip
mv root/zh_gen2oie_s2_aact_moie data
gdown 19R9uH73qLl9aFVJWqEzptQgn37Kcj01z
unzip zh_gen2oie_s1_aact_moie.zip
mv root/zh_gen2oie_s1_aact_moie data
rm -r root

bash train.sh zh_gen2oie_s1_aact_moie models/zh_gen2oie_s1_aact_moie_scratch data $DEVICE 10000
bash train.sh zh_gen2oie_s2_aact_moie models/zh_gen2oie_s2_aact_moie_scratch data $DEVICE 10000

## Uncomment below lines for running on TPU
# DEVICE=$TPU_NAME
# bash train.sh zh_gen2oie_s1_aact_moie gs://BUCKET_NAME/models/zh_gen2oie_s1_aact_moie_scratch gs://BUCKET_NAME/data $DEVICE 10000
# bash train.sh zh_gen2oie_s2_aact_moie gs://BUCKET_NAME/models/zh_gen2oie_s2_aact_moie_scratch gs://BUCKET_NAME/data $DEVICE 10000

### CHINESE Evaluation
Result with pre-trained model: (33.43 F1, 12.82 AUC)


In [ ]:
%%shell
LANGUAGE=zh
INPUT=carb/data/${LANGUAGE}_test.input
OUTPUT=${LANGUAGE}_test.output
DEVICE_NAME=0 ## GPU device number 0

## Download trained models
mkdir -p models
gdown 1gSJHJFmMsLFIgW-xJbI5RqB1q__LpNEU
unzip ${LANGUAGE}_gen2oie_s2_aact_moie.zip
mv root/${LANGUAGE}_gen2oie_s2_aact_moie models/
rm -r root
gdown 1zV0a-gK4z1RkTzl1h1kmQOCrfryYoamv
unzip ${LANGUAGE}_gen2oie_s1_aact_moie.zip
mv root/${LANGUAGE}_gen2oie_s1_aact_moie models/
rm -r root
mkdir -p models/${LANGUAGE}_label_rescore # label_rescore
gdown 1Yw-STBWAMff4NycHWHzBXmsichTI5I3r
mv ${LANGUAGE}_label_rescore.ckpt models/${LANGUAGE}_label_rescore

bash gen2oie_evaluate.sh ${LANGUAGE} models/${LANGUAGE}_gen2oie_s1_aact_moie models/${LANGUAGE}_gen2oie_s2_aact_moie models/${LANGUAGE}_label_rescore ${DEVICE_NAME} ${INPUT} ${OUTPUT}
python carb/carb.py --gold carb/data/gold/${LANGUAGE}_test.tsv --out /dev/null --allennlp ${LANGUAGE}_test.output.carb

## Run on TPU
# bash gen2oie_evaluate.sh ${LANGUAGE} gs://moie_bucket/models/${LANGUAGE}_gen2oie_s1_aact_moie gs://moie_bucket/models/${LANGUAGE}_gen2oie_s2_aact_moie models/${LANGUAGE}_label_rescore $TPU_NAME ${INPUT} ${OUTPUT}
